### Notebook con códigos de kaggle
Copiar notebook de kaggle de soluciones de competidores.

En este ejemplo, se copia uno que utiliza Light GBM

SOURCE: http://localhost:8888/lab/tree/Documents/GitHub/machine-learning-models-ds/discovery/m5/copy-kaggle-lightgbm.ipynb


**DUDA: aún revisando cómo hace el forecast a 28 días, que hace con los datos y con el modelo para poder obtener el forecast a ese nivel de granularidad**

In [1]:
import os
# set path root of repo
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('/')[:-1]
root_path = '/'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

root path:  /Users/joseortega/Documents/GitHub/forecasting-m5-dataset


In [2]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

### 0. Read original input files

In [3]:
folder_data = 'data/data_input_raw/'
df_cal = pd.read_csv(folder_data + 'calendar.csv')
df_eval = pd.read_csv(folder_data + 'sales_train_evaluation.csv')
df_price = pd.read_csv(folder_data + 'sell_prices.csv')
df_sample_output = pd.read_csv(folder_data + 'sample_submission.csv')

In [4]:
df_cal.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [5]:
# df con las ventas
df_eval.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [6]:
# exógena precio
df_price.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [7]:
# example output
df_sample_output.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# revisar unique id existentes
df_sample_output['id'].unique().shape # demasiados

(60980,)

### 1. Process data

#### Generar dataframes

In [9]:
# process calender
holiday = ['NewYear', 'OrthodoxChristmas', 'MartinLutherKingDay', 'SuperBowl', 'PresidentsDay', 'StPatricksDay', 'Easter', 'Cinco De Mayo', 'IndependenceDay', 'EidAlAdha', 'Thanksgiving', 'Christmas']
weekend = ['Saturday', 'Sunday']

df_cal['is_holiday_1'] = df_cal['event_name_1'].apply(lambda x : 1 if x in holiday else 0 )
df_cal['is_holiday_2'] = df_cal['event_name_1'].apply(lambda x : 1 if x in holiday else 0 )
df_cal['is_holiday'] = df_cal[['is_holiday_1','is_holiday_2']].max(axis=1)
df_cal['is_weekend'] = df_cal['weekday'].apply(lambda x : 1 if x in weekend else 0 )

df_cal = df_cal.drop(['weekday', 'wday', 'month', 'year', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2'], axis='columns')

In [10]:
# columns to delete
del_col = []
for x in range(1851):
    del_col.append('d_' + str(x+1))

In [11]:
df_eval.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [12]:
# eleminar columnas del dataframe de ventas
df_eval = df_eval.drop(del_col, axis='columns')
df_eval.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1852,d_1853,d_1854,d_1855,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,1,0,1,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,1,2,3,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,3,1,...,0,0,0,2,1,0,0,2,1,0


In [13]:
# transform ventas y dejarlo en formato ID, TIMESTAMP, Y
df_eval = df_eval.melt(['id','item_id','dept_id','cat_id','store_id','state_id'], var_name='d', value_name='qty')
print(df_eval.shape)
df_eval.head()

(2744100, 8)


,id,item_id,dept_id,cat_id,store_id,state_id,d,qty
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,1
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0


In [14]:
# merge ventas con variables exógenas calendario y con precios
df_eval = pd.merge(df_eval, df_cal, how='left', on='d')
df_eval = pd.merge(df_eval, df_price, how='left', on=['item_id', 'wm_yr_wk', 'store_id'])
df_eval.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,qty,date,wm_yr_wk,snap_CA,snap_TX,snap_WI,is_holiday_1,is_holiday_2,is_holiday,is_weekend,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0,2016-02-23,11604,0,0,0,0,0,0,0,8.26
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0,2016-02-23,11604,0,0,0,0,0,0,0,3.97
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0,2016-02-23,11604,0,0,0,0,0,0,0,2.97
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,1,2016-02-23,11604,0,0,0,0,0,0,0,4.64
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0,2016-02-23,11604,0,0,0,0,0,0,0,2.88


In [15]:
# generar dataframe para test
df_eval_test = df_eval.query('d == "d_1852"')
df_eval_test = df_eval_test[['id', 'store_id', 'item_id', 'dept_id', 'cat_id', 'state_id', 'd', 'qty', 'sell_price']]
df_eval_test['qty'] = df_eval_test['d'].apply(lambda x: int(x.replace(x, '0')))
df_eval_test.head()

,id,store_id,item_id,dept_id,cat_id,state_id,d,qty,sell_price
0,HOBBIES_1_001_CA_1_evaluation,CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA,d_1852,0,8.26
1,HOBBIES_1_002_CA_1_evaluation,CA_1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA,d_1852,0,3.97
2,HOBBIES_1_003_CA_1_evaluation,CA_1,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA,d_1852,0,2.97
3,HOBBIES_1_004_CA_1_evaluation,CA_1,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA,d_1852,0,4.64
4,HOBBIES_1_005_CA_1_evaluation,CA_1,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA,d_1852,0,2.88


In [16]:
# generar dataframe para test. Replicar valores Para los 28 días de predicciones
tmp_df = df_eval_test
for x in range(28):
    df_eval_test = pd.concat([df_eval_test, tmp_df])
    
df_eval_test = df_eval_test.reset_index(drop=True)

In [17]:
df_eval_test.shape

(884210, 9)

In [18]:
# list index day: "d_1942"
lst_d = []
i = 0
lst_index = df_eval_test.index
for x in lst_index:
    lst_d.append('d_' + str(((lst_index[i]) // 30490) + 1942))
    i = i + 1

In [19]:
# generar dataframe para test. Corección index
df_eval_test['d'] = lst_d

In [20]:
# generar dataframe para test. Append variables exógenas calendario y precios
df_eval_test = pd.merge(df_eval_test, df_cal, how='left', on='d')
df_eval_test = pd.merge(df_eval_test, df_price, how='left', on=['item_id', 'wm_yr_wk', 'store_id'])

#### gc

In [21]:
import gc
del tmp_df
gc.collect()

0

#### transformaciones datos - procesar variables categóricas

In [22]:
# transformar variables categóricas a binarias True / False
df_eval = pd.get_dummies(data=df_eval, columns=['dept_id', 'cat_id', 'store_id', 'state_id'])
df_eval_test = pd.get_dummies(data=df_eval_test, columns=['dept_id', 'cat_id', 'store_id', 'state_id'])

#### eliminar columnas

In [23]:
# eliminar columnas
df_eval_test = df_eval_test.drop(['sell_price_x', 'snap_CA', 'snap_TX', 'snap_WI'], axis='columns')
df_eval_test = df_eval_test.rename(columns={'sell_price_y': 'sell_price'})
df_eval = df_eval.drop(['snap_CA', 'snap_TX', 'snap_WI'], axis='columns')

### 2. Elegir variables y generar dataframe train y test

In [24]:
from sklearn.model_selection import train_test_split
target_col = 'qty'
exclude_cols = ['id', 'item_id', 'd', 'date', 'wm_yr_wk']
feature_cols = [col for col in df_eval.columns if col not in exclude_cols]
y = np.array(df_eval[target_col])
X = np.array(df_eval[feature_cols])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [25]:
# mostrar las features que se utilizaron com entrenamiento
df_eval[feature_cols].head()

,qty,is_holiday_1,is_holiday_2,is_holiday,is_weekend,sell_price,dept_id_FOODS_1,dept_id_FOODS_2,dept_id_FOODS_3,dept_id_HOBBIES_1,...,store_id_CA_4,store_id_TX_1,store_id_TX_2,store_id_TX_3,store_id_WI_1,store_id_WI_2,store_id_WI_3,state_id_CA,state_id_TX,state_id_WI
0,0,0,0,0,0,8.26,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
1,0,0,0,0,0,3.97,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
2,0,0,0,0,0,2.97,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
3,1,0,0,0,0,4.64,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
4,0,0,0,0,0,2.88,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False


In [26]:
# listado de features utilizadas
feature_cols

['qty',
 'is_holiday_1',
 'is_holiday_2',
 'is_holiday',
 'is_weekend',
 'sell_price',
 'dept_id_FOODS_1',
 'dept_id_FOODS_2',
 'dept_id_FOODS_3',
 'dept_id_HOBBIES_1',
 'dept_id_HOBBIES_2',
 'dept_id_HOUSEHOLD_1',
 'dept_id_HOUSEHOLD_2',
 'cat_id_FOODS',
 'cat_id_HOBBIES',
 'cat_id_HOUSEHOLD',
 'store_id_CA_1',
 'store_id_CA_2',
 'store_id_CA_3',
 'store_id_CA_4',
 'store_id_TX_1',
 'store_id_TX_2',
 'store_id_TX_3',
 'store_id_WI_1',
 'store_id_WI_2',
 'store_id_WI_3',
 'state_id_CA',
 'state_id_TX',
 'state_id_WI']

In [27]:
print('shape dataframes')
print('X_train: ', X_train.shape)
print('y_train: ', y_train.shape)

print('X_test: ', X_test.shape)
print('y_test: ', y_test.shape)

shape dataframes
X_train:  (2058075, 29)
y_train:  (2058075,)
X_test:  (686025, 29)
y_test:  (686025,)


### 3. Entrenar modelo

In [28]:
import lightgbm as lgb
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test)

In [29]:
### show dataset usado para entrenameinto
X_train

array([[0, 0, 0, ..., True, False, False],
       [0, 0, 0, ..., False, True, False],
       [0, 0, 0, ..., False, True, False],
       ...,
       [1, 0, 0, ..., True, False, False],
       [2, 0, 0, ..., True, False, False],
       [5, 1, 1, ..., False, False, True]], dtype=object)

In [30]:
params = {
        'n_jobs': -1,
        'boosting_type': 'gbdt',
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 64,
        "learning_rate" : 0.005,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 6,
        "bagging_seed" : 42,
        "verbosity" : 1,
        "seed": 42,
        }   

evals_result = {}

model = lgb.train(params, lgb_train, 5000, 
                      valid_sets=[lgb_eval], 
                      #verbose_eval=50, 
                      #evals_result=evals_result,
                      callbacks=[
                        lgb.early_stopping(stopping_rounds=3),
                        ]
                 )

[LightGBM] [Warning] Unknown parameter: bagging_frequency
[LightGBM] [Warning] Unknown parameter: bagging_frequency
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022248 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 379
[LightGBM] [Info] Number of data points in the train set: 2058075, number of used features: 29
[LightGBM] [Warning] Unknown parameter: bagging_frequency
[LightGBM] [Info] Start training from score 1.395218
Training until validation scores don't improve for 3 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

In [31]:
model

### 4. Predecir

DUDA: 
- la idea es hacer un forecast a 28 días a futuro, cómo se puede hacer para predecirlo? 
- Entrenar un modelo que prediga un step a futuro y luego de forma recursiva hacer las predicciones (ej como lo hace nixtla o skforecast)
- Entrenar 28 modelos para que cada step sea un modelo (esto considerando que se pasan todas las series juntas con el identificador de la serie, pero aqui son muchas series...)

In [32]:
pred = model.predict(df_eval_test[feature_cols])
pred

array([ 0.00992236, -0.00550599, -0.00290469, ...,  0.18129895,
        0.18129895,  0.18129895])

In [33]:
df_eval_test['pred_qty'] = pred

In [34]:
predictions = df_eval_test[['id', 'date', 'pred_qty']]
predictions.head()

,id,date,pred_qty
0,HOBBIES_1_001_CA_1_evaluation,2016-05-23,0.009922
1,HOBBIES_1_002_CA_1_evaluation,2016-05-23,-0.005506
2,HOBBIES_1_003_CA_1_evaluation,2016-05-23,-0.002905
3,HOBBIES_1_004_CA_1_evaluation,2016-05-23,-0.002896
4,HOBBIES_1_005_CA_1_evaluation,2016-05-23,-0.004594


In [35]:
# pivotear para obtener las predicciones para cada id los proximos 28 días
predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'pred_qty').reset_index()
predictions.head()

date,id,NaN,2016-05-23,2016-05-24,2016-05-25,2016-05-26,2016-05-27,2016-05-28,2016-05-29,2016-05-30,...,2016-06-10,2016-06-11,2016-06-12,2016-06-13,2016-06-14,2016-06-15,2016-06-16,2016-06-17,2016-06-18,2016-06-19
0,FOODS_1_001_CA_1_evaluation,0.028272,-0.008112,-0.008112,-0.008112,-0.008112,-0.008112,-0.014734,-0.014734,-0.008112,...,-0.008112,-0.014734,-0.014734,-0.008112,-0.008112,-0.008112,-0.008112,-0.008112,-0.014734,-0.014734
1,FOODS_1_001_CA_2_evaluation,0.006626,0.045767,0.045767,0.045767,0.045767,0.045767,0.060464,0.060464,0.045767,...,0.045767,0.060464,0.060464,0.045767,0.045767,0.045767,0.045767,0.045767,0.060464,0.060464
2,FOODS_1_001_CA_3_evaluation,0.397741,-0.004633,-0.004633,-0.004633,-0.004633,-0.004633,-0.003026,-0.003026,-0.004633,...,-0.004633,-0.003026,-0.003026,-0.004633,-0.004633,-0.004633,-0.004633,-0.004633,-0.003026,-0.003026
3,FOODS_1_001_CA_4_evaluation,-0.069504,0.004523,0.004523,0.004523,0.004523,0.004523,-0.008964,-0.008964,0.004523,...,0.004523,-0.008964,-0.008964,0.004523,0.004523,0.004523,0.004523,0.004523,-0.008964,-0.008964
4,FOODS_1_001_TX_1_evaluation,0.020710,-0.018721,-0.018721,-0.018721,-0.018721,-0.018721,-0.022239,-0.022239,-0.018721,...,-0.018721,-0.022239,-0.022239,-0.018721,-0.018721,-0.018721,-0.018721,-0.018721,-0.022239,-0.022239


In [36]:
predictions = predictions.drop(predictions.columns[1], axis=1)

In [37]:
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

In [38]:
x = 2744099 + 1 - 853720
df_val = df_eval[x:]

In [39]:
predictions_v = df_val[['id', 'date', 'qty']]
predictions_v = pd.pivot(predictions_v, index = 'id', columns = 'date', values = 'qty').reset_index()

In [40]:
predictions_v['id'] = predictions['id'].apply(lambda x: x.replace('evaluation', 'validation'))

In [41]:
predictions_v.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

In [42]:
predictions_concat = pd.concat([predictions, predictions_v], axis=0)

In [43]:
# output final
predictions_concat

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_evaluation,-0.008112,-0.008112,-0.008112,-0.008112,-0.008112,-0.014734,-0.014734,-0.008112,-0.008112,...,-0.008112,-0.014734,-0.014734,-0.008112,-0.008112,-0.008112,-0.008112,-0.008112,-0.014734,-0.014734
1,FOODS_1_001_CA_2_evaluation,0.045767,0.045767,0.045767,0.045767,0.045767,0.060464,0.060464,0.045767,0.045767,...,0.045767,0.060464,0.060464,0.045767,0.045767,0.045767,0.045767,0.045767,0.060464,0.060464
2,FOODS_1_001_CA_3_evaluation,-0.004633,-0.004633,-0.004633,-0.004633,-0.004633,-0.003026,-0.003026,-0.004633,-0.004633,...,-0.004633,-0.003026,-0.003026,-0.004633,-0.004633,-0.004633,-0.004633,-0.004633,-0.003026,-0.003026
3,FOODS_1_001_CA_4_evaluation,0.004523,0.004523,0.004523,0.004523,0.004523,-0.008964,-0.008964,0.004523,0.004523,...,0.004523,-0.008964,-0.008964,0.004523,0.004523,0.004523,0.004523,0.004523,-0.008964,-0.008964
4,FOODS_1_001_TX_1_evaluation,-0.018721,-0.018721,-0.018721,-0.018721,-0.018721,-0.022239,-0.022239,-0.018721,-0.018721,...,-0.018721,-0.022239,-0.022239,-0.018721,-0.018721,-0.018721,-0.018721,-0.018721,-0.022239,-0.022239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516_TX_2_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
30486,HOUSEHOLD_2_516_TX_3_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000
30487,HOUSEHOLD_2_516_WI_1_validation,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
30488,HOUSEHOLD_2_516_WI_2_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
